In [1]:
import numpy as np


def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "/mnt/Cache/data/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((3601, 500, 1), (3601,), (1320, 500, 1), (1320,))

In [2]:
from model import *

2021-11-25 11:11:15.146824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-25 11:11:15.146837: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    n_classes=n_classes,
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 1)]     0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 500, 1)       2           input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 500, 1)       7169        layer_normalization[0][0]        
                                                                 layer_normalization[0][0]        
__________________________________________________________________________________________________
dropout (Dropout)               (None, 500, 1)       0           multi_head_attention[0][0]   

2021-11-25 11:11:15.955143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 11:11:15.955368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-25 11:11:15.955398: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-11-25 11:11:15.955425: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-11-25 11:11:15.955452: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [ ]:
model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=16,
    callbacks=callbacks,
)

Epoch 1/100


2021-11-25 11:11:16.244297: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


180/180 [==============================] - 225s 1s/step - loss: 0.9037 - sparse_categorical_accuracy: 0.5267 - val_loss: 0.6789 - val_sparse_categorical_accuracy: 0.6061
Epoch 2/100
180/180 [==============================] - 225s 1s/step - loss: 0.7599 - sparse_categorical_accuracy: 0.5965 - val_loss: 0.6293 - val_sparse_categorical_accuracy: 0.6422
Epoch 3/100
180/180 [==============================] - 228s 1s/step - loss: 0.6790 - sparse_categorical_accuracy: 0.6444 - val_loss: 0.5997 - val_sparse_categorical_accuracy: 0.6602
Epoch 4/100
126/180 [====================>.........] - ETA: 1:03 - loss: 0.6503 - sparse_categorical_accuracy: 0.6553

In [ ]:
model.evaluate(x_test, y_test, verbose=1)

In [ ]:
model.save("/mnt/Cache/data/player-pricing/models/transformer-forda")